## Demostration of the ```add_ontology_subtree``` endpoint

We demonstrate the ability to add any ontology term to the MIRA DKG given its curie. We also provide the option to add all taxonomy terms in the subtree of the specified taxonomy term to add. Currently we only support the addition of **ncbitaxonomy** terms. This demonstration will make use of the white-tailed deer ncbitaxonomy term: https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?mode=Info&id=9874&lvl=3&lin=f&keep=1&srchmode=1&unlock.

In [19]:
import requests

## Define the ncbitaxonomy curie to add.
We first only add term itself and no terms under its subtree to the DKG.

In [2]:
# The ncbitaxonomy term with identifier 9874 corresponds to "Odocoileus Virginianus" which represents white-tailed deer. 
curie = 'ncbitaxon:9874'
add_subtree = False
params = {"curie": curie, "add_subtree": False}

In [18]:
response = requests.post("http://mira-epi-dkg-lb-dc1e19b273dedaa2.elb.us-east-1.amazonaws.com:8771/api/add_ontology_subtree", params=params)
response.status_code

200

## Search for the added node in the DKG

We then use the ```search``` endpoint to look for the newly added node that represents white-tailed deer. Looking at the ontological structure of the white-tailed deer term: https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?mode=Tree&id=9874&lvl=3&lin=f&keep=1&srchmode=1&unlock, we see that there exists a child term in the subtree for white-tailed deer with curie ```ncbitaxon:9875``` that corresponds to Virginia white-tailed deer. We also search for Virginia white-tailed deer to showcase that no subtree terms have been added.

In [4]:
search_params = {
    "q": "white-tailed deer",
    "limit": 1,
    "offset": 0,
    "prefixes": "ncbitaxon",
    "wikidata_fallback": False
}
response = requests.get("http://mira-epi-dkg-lb-dc1e19b273dedaa2.elb.us-east-1.amazonaws.com:8771/api/search", params=search_params)
print(response.json()[0]["synonyms"][0]["value"])
print()
response.json()

white-tailed deer



[{'id': 'ncbitaxon:9874',
  'name': 'Odocoileus virginianus',
  'type': 'class',
  'obsolete': False,
  'synonyms': [{'value': 'white-tailed deer', 'type': ''}],
  'xrefs': [{'id': 'gc_id:1', 'type': ''}],
  'labels': [],
  'properties': {'has_rank': ['NCBITaxon:species']}}]

In [6]:
search_params = {
    "q": "Virginia white-tailed deer",
    "limit": 1,
    "offset": 0,
    "prefixes": "ncbitaxon",
    "wikidata_fallback": False
}
response = requests.get("http://mira-epi-dkg-lb-dc1e19b273dedaa2.elb.us-east-1.amazonaws.com:8771/api/search", params=search_params)
print(response.json())

[]


## Add the ontology subtree full of terms under the specified term

We use the same exact curie representing white-tailed deer and endpoint with the only difference being that we set the ```add_subtree``` flag to True.

In [10]:
params = {"curie": curie, "add_subtree": True}
response = requests.post("http://mira-epi-dkg-lb-dc1e19b273dedaa2.elb.us-east-1.amazonaws.com:8771/api/add_ontology_subtree", params=params)
response.status_code

200

## Relationship info has been added

Using the ```search``` endpoint for Virginia white-tailed deer now returns results

In [12]:
search_params = {
    "q": "Virginia white-tailed deer",
    "limit": 1,
    "offset": 0,
    "prefixes": "ncbitaxon",
    "wikidata_fallback": False
}
response = requests.get("http://mira-epi-dkg-lb-dc1e19b273dedaa2.elb.us-east-1.amazonaws.com:8771/api/search", params=search_params)
print(response.json()[0]["synonyms"][0]["value"])
print()
response.json()

Virginia white-tailed deer



[{'id': 'ncbitaxon:9875',
  'name': 'Odocoileus virginianus virginianus',
  'type': 'class',
  'obsolete': False,
  'synonyms': [{'value': 'Virginia white-tailed deer', 'type': ''}],
  'xrefs': [{'id': 'gc_id:1', 'type': ''}],
  'labels': [],
  'properties': {'has_rank': ['NCBITaxon:subspecies']}}]

## Show the relation between white-tailed deer and Virginia white-tailed deer

We then use the ```relations``` endpoint to query for the ```subclass_of``` relation between source node ```ncbitaxon:9875``` and target node ```ncbitaxon:9874```.


In [16]:
relations_json = {
  "source_curie": "ncbitaxon:9875",
  "target_curie": "ncbitaxon:9874",
  "relations": "subclass_of"
}

response = requests.post("http://mira-epi-dkg-lb-dc1e19b273dedaa2.elb.us-east-1.amazonaws.com:8771/api/relations", json=relations_json)
response.json()

[{'subject': 'ncbitaxon:9875',
  'predicate': 'rdfs:subClassOf',
  'object': 'ncbitaxon:9874'}]